# 🧠 Agentic Analytics Part 2: CrewAI + MCP Implementation

This notebook demonstrates a simplified **agentic data pipeline** using [CrewAI](https://github.com/joaomdmoura/crewai) and MCP. It showcases how multiple AI agents can work together to perform real-world analytics tasks—in this case, pulling data using MCP and visualizing the Iris dataset.

We simulate:
- 🛰️ A **Data Ingestion Agent**
- 📊 A **Visualization Agent**

These agents collaborate in a structured pipeline, allowing you to experience how agent roles and task delegation work in practice—**without glue code or manual orchestration**.

---

👤 **Author**: [Dr. K.M. Mohsin](https://www.linkedin.com/in/km-mohsin/)  
Follow me on LinkedIn to explore more around **Agentic AI, CrewAI, LangGraph, and real-world LLM applications in analytics, automation, and education.**

## ✅ Step 1: Install Dependencies and LLM Key

In [ ]:
!pip install -U crewai openai pandas matplotlib tabulate python-dotenv

### Place .env file and save your LLM Key

In [ ]:
# Load your OpenAI API key securely
import os
from dotenv import load_dotenv

# Option 1 (Quick, not secure)
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Option 2 (Preferred: .env file with OPENAI_API_KEY=sk-...)
load_dotenv()


## 🧠 Step 2: Define Tools using MCP `@tool`

1.   Get Weather Data using API (no authentication required, it's open)
2.   Plot Weather Data



In [ ]:
from crewai.tools import tool

@tool("Get Weather Data")
def get_weather_data(city: str) -> str:
    """Fetches hourly temperature data for a given city (currently supports 'Houston')."""
    import requests
    import pandas as pd
    coords = {"houston": (29.7604, -95.3698)}
    lat, lon = coords.get(city.lower(), (29.7604, -95.3698))
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m"
    res = requests.get(url).json()
    df = pd.DataFrame({
        "time": res["hourly"]["time"],
        "temp": res["hourly"]["temperature_2m"]
    })
    return df.to_json()

@tool("Plot Weather Data")
def plot_weather_data(df_json: str) -> str:
    """Generates a line plot of temperature over time."""
    import pandas as pd
    import matplotlib.pyplot as plt
    df = pd.read_json(df_json)
    df.plot(x='time', y='temp', title='Hourly Temperature (Houston)', legend=False)
    plt.xlabel('Time')
    plt.ylabel('Temp (°C)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("weather_plot.png")
    return "weather_plot.png"



## 👥 Step 3: Define Agents

In [ ]:
from crewai import Agent

def build_agents():
    ingestion = Agent(
        name="WeatherIngestionAgent",
        role="Fetches live weather data for a city",
        goal="Get hourly temperature data for Houston using an external API.",
        backstory="This agent connects to public weather APIs to fetch real-time data for analysis.",
        tools=[get_weather_data]
    )
    visualization = Agent(
        name="VizAgent",
        role="Plots weather data",
        goal="Generate a temperature-over-time plot for Houston.",
        backstory="This agent specializes in creating informative visualizations from structured datasets.",
        tools=[plot_weather_data]
    )
    return ingestion, visualization


## 🛠️ Step 4: Run the Crew (Multiple Agents on Multiple Tasks)

In [ ]:

from crewai import Crew, Task

ingestion_agent, viz_agent = build_agents()

crew = Crew(
    agents=[ingestion_agent, viz_agent],
    tasks=[
        Task(
            agent=ingestion_agent,
            description="Fetch hourly temperature forecast for Houston from a weather API and return it as JSON.",
            expected_output="A JSON string with hourly time and temperature data for Houston.",
        ),
        Task(
            agent=viz_agent,
            description="Generate a line plot of hourly temperature using the provided JSON data.",
            expected_output="A file named 'weather_plot.png' containing the temperature line chart.",
        )
    ]
)

crew.kickoff()
